In [65]:
from torch_geometric.datasets import TUDataset
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchsummary import summary
import torch.nn.functional as F
from torch_geometric.transforms import TargetIndegree

import time
from torch_geometric.data import Data, DataLoader
import numpy.linalg as LA
import networkx as nx
from torch_geometric.utils import to_networkx


if torch.cuda.is_available() is False:
    raise Exception("GPU device not found, runtime environment should be set to GPU")
print(f"Using GPU device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

Using GPU device: GeForce RTX 2080 Ti


In [238]:
dataset=TUDataset('data', 'PROTEINS', transform=TargetIndegree()) #saves normalized degree as edge_attr
print(f'Number of graphs: {len(dataset)}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of node features: {dataset.num_node_features}')

Number of graphs: 1113
Number of classes: 2
Number of node features: 3


In [239]:
dataset = dataset.shuffle()
train_dataset = dataset[150:]
test_dataset = dataset[:150]
train_loader=DataLoader(train_dataset,batch_size=100)
test_loader=DataLoader(test_dataset,batch_size=150)
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 963
Number of test graphs: 150


In [211]:
from torch_geometric.nn import ChebConv
from torch_geometric.nn import BatchNorm
from torch_geometric.nn import global_mean_pool
class Cheb(nn.Module):
    def __init__(self, in_channel,hidden,K):
        super().__init__()
        self.conv1 = ChebConv(in_channel,hidden,K)
        self.bn1 = BatchNorm(hidden)
        self.dense1 = nn.Linear(hidden,2)
        self.relu = nn.ReLU()
        self.hidden=hidden
        self.history= None
        
    def forward(self,data):
        edge_index=data.edge_index
        h=data.x
        h = self.conv1(h, edge_index)
        h = self.bn1(h)
        h = self.relu(h)
        h = global_mean_pool(h, data.batch)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.dense1(h)
        
        return h

In [215]:
from torch_geometric.nn import ARMAConv
class ARMA(torch.nn.Module):
    def __init__(self,in_channel,hidden,K,N):
        super().__init__()
        self.conv1 = ARMAConv(in_channel,hidden,num_stacks=K, num_layers=N)
        self.bn1 = BatchNorm(hidden)
        self.dense1 = nn.Linear(hidden,2)
        self.relu = nn.ReLU()
        self.hidden=hidden
        self.history= None
        
    def forward(self,data):
        edge_index=data.edge_index
        h=data.x
        h = self.conv1(h, edge_index)
        h = self.bn1(h)
        h = self.relu(h)
        h = global_mean_pool(h, data.batch)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.dense1(h)
        
        return h

In [241]:
from torch_geometric.nn import SplineConv
class Spline(torch.nn.Module):
    def __init__(self,in_channel,hidden,kernel):
        super().__init__()
        self.conv1 = SplineConv(in_channel, hidden, dim=1, kernel_size=kernel)
        self.bn1 = BatchNorm(hidden)
        self.dense1 = torch.nn.Linear(hidden,2)

    def forward(self, data):
        h = F.relu(self.bn1(self.conv1(data.x, data.edge_index,data.edge_attr)))
        h = global_mean_pool(h, data.batch)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.dense1(h)
        return h



In [172]:
def training_loop(EPOCHS, MODEL, OPTIMIZER, device, train_loader, test_loader):
    summary(MODEL)
    
    if torch.cuda.is_available() is False:
        raise Exception("GPU device not found, runtime environment should be set to GPU")
    print(f"Using GPU device: {torch.cuda.get_device_name(device)}")
    
    
    criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
    history = {'train_loss': [], 'train_accuracy': [], 'test_loss': [],'test_accuracy': []
               ,'time':None}

    for epoch in range(EPOCHS):
        start=time.time()
        
        temp_loss = 0
        correct=0
        graphs=0
        for step,data in enumerate(train_loader):
            data.to(device)
            OPTIMIZER.zero_grad()  # Clear gradients.
            y_out = MODEL(data) # Perform a single forward pass.
            loss = criterion(y_out, data.y)  # Compute the loss solely based on the training nodes.
            loss.backward()  # Derive gradients.
            OPTIMIZER.step()  # Update parameters based on gradients.
            
            temp_loss=temp_loss+loss.detach().item()
            pred = y_out.argmax(dim=1)  # Use the class with highest probability.
            correct =correct + int((pred == data.y).sum())
            graphs = graphs + data.num_graphs
            
        train_acc=float("{:.2f}".format(correct / graphs))
        train_loss=float("{:.2f}".format(temp_loss/(step+1)))
        
         
        temp_loss = 0
        correct=0
        graphs=0  
        for step,data in enumerate(test_loader):
            data.to(device)
            y_out =MODEL(data)
            loss = criterion(y_out,  data.y) 
            
            temp_loss =  temp_loss + loss.detach().item()
            pred = y_out.argmax(dim=1)  # Use the class with highest probability.
            correct =correct + int((pred == data.y).sum())
            graphs = graphs + data.num_graphs
        
        test_acc=float("{:.2f}".format(correct / graphs))
        test_loss=float("{:.2f}".format(temp_loss/(step+1)))
        
        end=time.time()
        
        print(f"Epoch: {epoch} | Train loss: {train_loss} | Train accuracy: {train_acc}  | Test loss: {test_loss} | Test accuracy: {test_acc}| Time: {end-start}")
        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_acc)
        history['test_accuracy'].append(test_acc)
        history['test_loss'].append(test_loss)
        history['time']=end-start
    return history

In [214]:
##TODOtrain
device = torch.cuda.current_device()
MODEL_Cheb=Cheb(3,30,30).to(device)
OPTIMIZER =  torch.optim.Adam(MODEL_Cheb.parameters(), lr=0.003)   
EPOCHS = 50

Cheb_history = training_loop(EPOCHS, MODEL_Cheb, OPTIMIZER, device,train_loader,test_loader)

Layer (type:depth-idx)                   Param #
├─ChebConv: 1-1                          2,730
├─BatchNorm: 1-2                         --
|    └─BatchNorm1d: 2-1                  60
├─Linear: 1-3                            62
├─ReLU: 1-4                              --
Total params: 2,852
Trainable params: 2,852
Non-trainable params: 0
Using GPU device: GeForce RTX 2080 Ti
Epoch: 0 | Train loss: 0.68 | Train accuracy: 0.63  | Test loss: 0.65 | Test accuracy: 0.65| Time: 0.2850046157836914
Epoch: 1 | Train loss: 0.64 | Train accuracy: 0.64  | Test loss: 0.63 | Test accuracy: 0.65| Time: 0.2621893882751465
Epoch: 2 | Train loss: 0.61 | Train accuracy: 0.69  | Test loss: 0.61 | Test accuracy: 0.71| Time: 0.27790069580078125
Epoch: 3 | Train loss: 0.59 | Train accuracy: 0.71  | Test loss: 0.6 | Test accuracy: 0.73| Time: 0.28606081008911133
Epoch: 4 | Train loss: 0.58 | Train accuracy: 0.71  | Test loss: 0.6 | Test accuracy: 0.71| Time: 0.26187562942504883
Epoch: 5 | Train loss: 0.57 | T

In [223]:
device = torch.cuda.current_device()
MODEL_ARMA=ARMA(3,K=3,N=1, hidden=30).to(device)
OPTIMIZER =  torch.optim.Adam(MODEL_ARMA.parameters(), lr=0.003)   
EPOCHS = 50

ARMA_history = training_loop(EPOCHS, MODEL_ARMA, OPTIMIZER, device,train_loader,test_loader)

Layer (type:depth-idx)                   Param #
├─ARMAConv: 1-1                          3,330
├─BatchNorm: 1-2                         --
|    └─BatchNorm1d: 2-1                  60
├─Linear: 1-3                            62
├─ReLU: 1-4                              --
Total params: 3,452
Trainable params: 3,452
Non-trainable params: 0
Using GPU device: GeForce RTX 2080 Ti
Epoch: 0 | Train loss: 0.68 | Train accuracy: 0.59  | Test loss: 0.67 | Test accuracy: 0.61| Time: 0.12637710571289062
Epoch: 1 | Train loss: 0.67 | Train accuracy: 0.59  | Test loss: 0.67 | Test accuracy: 0.61| Time: 0.1127007007598877
Epoch: 2 | Train loss: 0.67 | Train accuracy: 0.59  | Test loss: 0.66 | Test accuracy: 0.61| Time: 0.10304117202758789
Epoch: 3 | Train loss: 0.67 | Train accuracy: 0.59  | Test loss: 0.66 | Test accuracy: 0.61| Time: 0.10242152214050293
Epoch: 4 | Train loss: 0.67 | Train accuracy: 0.6  | Test loss: 0.66 | Test accuracy: 0.63| Time: 0.09877300262451172
Epoch: 5 | Train loss: 0.66 |

In [253]:
device = torch.cuda.current_device()
MODEL_Spline=Spline(3,hidden=30,kernel=10).to(device)
OPTIMIZER =  torch.optim.Adam(MODEL_Spline.parameters(), lr=0.03)   
EPOCHS = 50

Spline_history = training_loop(EPOCHS, MODEL_Spline, OPTIMIZER, device,train_loader,test_loader)

Layer (type:depth-idx)                   Param #
├─SplineConv: 1-1                        1,020
├─BatchNorm: 1-2                         --
|    └─BatchNorm1d: 2-1                  60
├─Linear: 1-3                            62
Total params: 1,142
Trainable params: 1,142
Non-trainable params: 0
Using GPU device: GeForce RTX 2080 Ti
Epoch: 0 | Train loss: 0.66 | Train accuracy: 0.62  | Test loss: 0.65 | Test accuracy: 0.61| Time: 0.1609508991241455
Epoch: 1 | Train loss: 0.62 | Train accuracy: 0.66  | Test loss: 0.63 | Test accuracy: 0.65| Time: 0.14517998695373535
Epoch: 2 | Train loss: 0.6 | Train accuracy: 0.67  | Test loss: 0.6 | Test accuracy: 0.7| Time: 0.14477276802062988
Epoch: 3 | Train loss: 0.58 | Train accuracy: 0.7  | Test loss: 0.62 | Test accuracy: 0.71| Time: 0.1489267349243164
Epoch: 4 | Train loss: 0.57 | Train accuracy: 0.7  | Test loss: 0.6 | Test accuracy: 0.71| Time: 0.14553546905517578
Epoch: 5 | Train loss: 0.56 | Train accuracy: 0.71  | Test loss: 0.61 | Test ac